<a href="https://colab.research.google.com/github/GrkRise/GenAI_labs/blob/main/GenAI_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лекция 2: "Агент-Аналитик" - Практическая работа

Этот ноутбук содержит полный код для создания "Агента-Аналитика" с использованием `llama-index`. Мы пройдем все шаги: от настройки окружения до извлечения структурированных требований и поиска противоречий в документах.



## Шаг 0: Подготовка рабочего окружения

Сначала установим все необходимые библиотеки. Нам понадобятся:
- `llama-index`: Основной фреймворк.
- `llama-index-llms-groq`: Интеграция с быстрыми и бесплатными моделями от Groq.
- `llama-index-embeddings-huggingface`: Для использования бесплатных локальных моделей эмбеддингов.
- `pypdf`, `reportlab`: Для работы с PDF-файлами.
- `sentence-transformers`: Зависимость для модели эмбеддингов.

In [ ]:
# Устанавливаем библиотеки в "тихом" режиме
!pip install -q llama-index llama-index-llms-groq llama-index-embeddings-huggingface pypdf reportlab sentence-transformers llama-index-program-openai

# Импортируем все необходимые модули
import os
import shutil
from getpass import getpass
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.program.openai import OpenAIPydanticProgram # Работает с любой LLM, не только OpenAI
from pydantic import BaseModel, Field
from typing import List
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter


### Настройка API ключа

Для работы нам понадобится бесплатный API-ключ от Groq.
1.  Зайдите на [https://console.groq.com/keys](https://console.groq.com/keys)
2.  Создайте и скопируйте бесплатный API-ключ.
3.  Вставьте его в поле ввода, которое появится после запуска следующей ячейки.

In [ ]:
print("Пожалуйста, введите ваш Groq API ключ:")
groq_api_key = getpass()

os.environ["GROQ_API_KEY"] = groq_api_key
print("✅ Groq API ключ успешно установлен.")

## Шаг 0.1: Настройка моделей (LLM и Embeddings)

Теперь мы настроим две ключевые модели в `Settings`, чтобы `llama-index` использовал их по умолчанию:
1.  **LLM (Языковая модель):** `llama3-70b-8192` от Groq. Она будет генерировать текст и заполнять наши Pydantic-модели.
2.  **Embed Model:** `BAAI/bge-small-en-v1.5` от HuggingFace. Эта модель будет работать **локально** прямо в Colab, чтобы превращать текст в векторы (эмбеддинги) для семантического поиска. Это бесплатно и не требует API-ключей.

In [ ]:
# Настраиваем LLM от Groq
Settings.llm = Groq(model="llama-3.3-70b-versatile", api_key=groq_api_key)

# Настраиваем локальную модель для эмбеддингов
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

print("✅ LLM (Groq) и Embed Model (HuggingFace) успешно настроены.")

## Шаг 0.5: Создание тестовых документов

Чтобы сделать ноутбук полностью самодостаточным, мы программно создадим папку `project_docs` и три файла с "сырыми" требованиями, которые наш агент будет анализировать.

In [ ]:
def create_dummy_pdf(file_path, text_content):
    """Вспомогательная функция для создания простого PDF-файла."""
    c = canvas.Canvas(file_path, pagesize=letter)
    width, height = letter
    text_object = c.beginText(50, height - 50); text_object.setFont("Helvetica", 12)
    for line in text_content.split('\n'): text_object.textLine(line)
    c.drawText(text_object); c.save()

# Контент для наших документов
pdf_content = "Проект: Smart Notes App. Описание: Приложение для создания и управления заметками. Пользователи должны иметь возможность делиться своими заметками с другими."
chat_log_content = "Алекс: Клиент настаивает на входе через Google. Как пользователь, я хочу иметь возможность делиться заметками, чтобы мои коллеги могли их комментировать."
features_md_content = "# План фичей v1.0\n## Аутентификация\n- [x] Логин по Email/паролю\n**Примечание:** На первом этапе делаем только вход по Email."

# Создаем папку и файлы
if os.path.exists("project_docs"): shutil.rmtree("project_docs")
os.makedirs("project_docs")
create_dummy_pdf("project_docs/brief.pdf", pdf_content)
with open("project_docs/chat_log.txt", "w", encoding='utf-8') as f: f.write(chat_log_content)
with open("project_docs/features.md", "w", encoding='utf-8') as f: f.write(features_md_content)

print("✅ Созданы 3 документа в папке 'project_docs'.")

## Шаги 1 и 2: Загрузка и Индексация

Теперь самое интересное. Мы используем `SimpleDirectoryReader` для загрузки всех документов из папки. Затем `VectorStoreIndex` превратит их в семантически доступную базу знаний, используя настроенную нами локальную embedding-модель.

In [ ]:
# Загружаем все документы из папки
documents = SimpleDirectoryReader("./project_docs").load_data()

# Создаем векторный индекс. На этом шаге происходит вся "магия" векторизации.
index = VectorStoreIndex.from_documents(documents)

print(f"✅ Документы загружены и проиндексированы локально. Создан индекс.")

## Шаг 3: Определение структуры вывода (Pydantic)

Мы хотим получать результат не в виде простого текста, а в виде структурированных Python-объектов. Для этого мы описываем "контракт" или "форму" для LLM с помощью Pydantic-моделей.

**Важно:** Каждый класс должен иметь `docstring` (док-строку `"""..."""`), чтобы LLM понимала общую цель создаваемого объекта.

In [ ]:
class AcceptanceCriterion(BaseModel):
    """Модель для одного критерия приемки."""
    criterion: str = Field(description="Текст критерия приемки.")

class UserStory(BaseModel):
    """Модель для полной пользовательской истории с ее критериями."""
    story: str = Field(description="Полный текст User Story в формате 'As a [user], I want [goal], so that [benefit]'.")
    criteria: List[AcceptanceCriterion] = Field(description="Список критериев приемки для этой истории.")

class RequirementsAnalysis(BaseModel):
    """Контейнер данных для хранения результатов анализа требований. Содержит список всех найденных пользовательских историй."""
    user_stories: List[UserStory] = Field(description="Список всех пользовательских историй, найденных в документах.")

print("✅ Pydantic-модели определены.")

## WOW-МОМЕНТ 1: Извлечение User Stories - Попытка №1 (Наивный подход)

Сейчас мы попробуем извлечь User Stories, используя "наивный" промпт. Мы дадим модели творческую свободу и разрешим ей "додумывать" информацию на основе контекста. Давайте посмотрим, что из этого получится.

In [ ]:
# Наивный промпт с разрешением "додумывать"
naive_prompt_template_str = """
Проанализируй предоставленный контекст из проектных документов.
Извлеки все пользовательские истории (user stories) и для каждой из них
сформулируй по крайней мере 2-3 релевантных критерия приемки.
Если история неполная, додумай ее на основе контекста.
"""

# Создаем программу с наивным промптом
naive_program = OpenAIPydanticProgram.from_defaults(
    output_cls=RequirementsAnalysis,
    prompt_template_str=naive_prompt_template_str,
    llm=Settings.llm,
    verbose=True
)

# Извлекаем релевантный контекст
query_engine = index.as_query_engine()
retrieved_nodes = query_engine.retrieve("Все пользовательские истории и требования")
context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])

# Запускаем программу для извлечения
response_naive = naive_program(context_str=context_str)

### Результат Попытки №1

Выведем результат, который сгенерировала модель.

In [ ]:
if response_naive and response_naive.user_stories:
    for i, story in enumerate(response_naive.user_stories, 1):
        print(f"История #{i}:")
        print(f"  - История: {story.story}")
        print("  - Критерии приемки:")
        for j, criterion in enumerate(story.criteria, 1):
            print(f"    {j}. {criterion.criterion}")
        print("-" * 20)
else:
    print("Модели не удалось извлечь User Stories.")

### Анализ результата: Классическая "галлюцинация"!

**Обратите внимание!** Модель сгенерировала идеально структурированные User Stories, но их содержание **не имеет ничего общего** с нашими документами про "Smart Notes App". Она придумала истории для вымышленного интернет-магазина.

**Почему это произошло?**
1.  **Слабый контекст:** Наши документы были очень короткими.
2.  **Излишняя свобода:** Наш промпт со словом "додумай" послужил разрешением на "творчество".
3.  **Итог:** Не найдя достаточно информации, модель обратилась к своим внутренним знаниям и сгенерировала самый типичный пример, который знает.

**Это важнейший урок:** нельзя слепо доверять выводу LLM. Теперь давайте это исправим.

## WOW-МОМЕНТ 1: Попытка №2 (Строгий промпт-инжиниринг)

Теперь мы напишем более строгий промпт. Мы явно **запретим** модели придумывать информацию и потребуем основываться **исключительно** на предоставленном тексте. Это ключевая техника для повышения надежности AI-агентов.

In [ ]:
# Строгий промпт с явными запретами
strict_prompt_template_str = """
Проанализируй предоставленный контекст из проектных документов.
Твоя задача — извлечь пользовательские истории (user stories), которые ЯВНО упомянуты в тексте.
Для каждой извлеченной истории сформулируй 2-3 релевантных критерия приемки, основываясь СТРОГО на контексте.
**ЗАПРЕЩЕНО:** Придумывать или генерировать истории, которых нет в тексте. Если в тексте нет явных user stories, верни пустой список.
"""

# Создаем новую программу со строгим промптом
strict_program = OpenAIPydanticProgram.from_defaults(
    output_cls=RequirementsAnalysis,
    prompt_template_str=strict_prompt_template_str,
    llm=Settings.llm,
    verbose=True
)

# Контекст у нас уже есть из предыдущего шага, можем использовать его повторно
response_strict = strict_program(context_str=context_str)

### Результат Попытки №2

Смотрим, что получилось со строгими правилами.

In [ ]:
if response_strict and response_strict.user_stories:
    for i, story in enumerate(response_strict.user_stories, 1):
        print(f"История #{i}:")
        print(f"  - История: {story.story}")
        print("  - Критерии приемки:")
        for j, criterion in enumerate(story.criteria, 1):
            print(f"    {j}. {criterion.criterion}")
        print("-" * 20)
else:
    print("✅ Модель не нашла явных User Stories в тексте. Это ПРАВИЛЬНЫЙ результат, так как она последовала нашему запрету на галлюцинации.")

## WOW-МОМЕНТ 2: Поиск противоречий

Теперь, когда мы научились контролировать агента, перейдем к аналитической задаче. Мы попросим его найти конфликты в требованиях, используя обычный текстовый запрос.

In [ ]:
text_query_engine = index.as_query_engine()

contradiction_prompt = """
Внимательно сравни требования к процессу аутентификации пользователя из всех документов.
Существуют ли между ними какие-либо противоречия?
Если да, четко опиши, в чем заключается противоречие, и укажи, в каких документах содержится конфликтующая информация.
"""

response_text = text_query_engine.query(contradiction_prompt)

### Результат (аналитический ответ)

In [ ]:
print(response_text)

# Практическое задание для самостоятельной работы

## Легенда

Прошла неделя после вашего анализа. Клиенту очень понравился ваш подход к автоматическому извлечению требований, и он считает его "магией". Вдохновившись, он прислал вам новую порцию документов с уточнениями к первому спринту и идеями для второго.

Ваша задача — обновить и расширить вашего "Агента-Аналитика", чтобы обработать новые данные, извлечь из них еще больше полезной информации и подготовить сводку для команды разработки.

---

## Входные данные (Новые документы от клиента)

Вам необходимо программно создать новую папку `project_docs_sprint2` и поместить в нее три новых файла со следующим содержимым:

### 1. `meeting_minutes.txt` (Протокол встречи)
```text
Обсуждение фичей 2-го спринта.
Ключевое: пользователи жалуются, что в заметках сложно ориентироваться. Предлагаю добавить систему тегов. Пользователь должен иметь возможность прикрепить к заметке несколько тегов (например, #работа, #идеи) и потом фильтровать все заметки по этим тегам. Это главная фича.
Мария: Хорошая идея. Также клиент упомянул, что хотел бы иметь возможность экспортировать все свои данные в JSON-файл в один клик, чтобы не бояться их потерять.
```

### 2. `feature_update_email.md` (Письмо от менеджера)
```markdown
### Обновление по аутентификации

Привет! Мы обсудили с клиентом противоречие, которое ты нашел. Он решил, что нам нужны ОБА способа входа.

**ИТОГО:** Система должна поддерживать вход как через **Google**, так и через **Email/пароль**. Это требование с высоким приоритетом.
```

### 3. `technical_constraints.pdf` (Технические ограничения)
*Используйте функцию `create_dummy_pdf` из ноутбука, чтобы создать PDF-файл с этим текстом:*
```text
Нефункциональные требования (НФТ):
1. Безопасность: Система должна соответствовать базовым принципам GDPR. Все пользовательские данные должны храниться в зашифрованном виде.
2. Производительность: Время ответа на любой запрос пользователя не должно превышать 500 миллисекунд.
```
---

## Ваша задача

**1. Адаптация кода:**
*   Модифицируйте код из лекции, чтобы он читал документы из новой папки `project_docs_sprint2`.

**2. Расширение модели данных (Pydantic):**
*   Ваша текущая модель `RequirementsAnalysis` умеет извлекать только User Stories. Вам нужно ее расширить, чтобы она могла хранить и другие типы требований.
*   Создайте новую Pydantic-модель `NonFunctionalRequirement` с полями `requirement: str` и `category: str` (например, "Безопасность", "Производительность").
*   Добавьте в основную модель `RequirementsAnalysis` два новых поля:
    *   `nfrs: List[NonFunctionalRequirement]` для хранения списка нефункциональных требований.
    *   `updates_summary: str` для хранения краткой сводки об изменениях.

**3. Промпт-инжиниринг (самая важная часть!):**
*   Напишите **новый, комплексный промпт** для вашей Pydantic-программы. Этот промпт должен давать агенту команду извлечь из всех документов **сразу три типа информации за один вызов**:
    1.  Все **User Stories** (включая новые, про теги и экспорт).
    2.  Все **нефункциональные требования** (про GDPR и производительность).
    3.  Краткую **сводку (summary)** об основных изменениях в требованиях (например, про обновление логики аутентификации).

**4. Запуск и анализ:**
*   Запустите обновленный код.
*   Проанализируйте полученный структурированный вывод. Удалось ли агенту корректно извлечь всю информацию?

---
## Что нужно сдать на проверку

1.  **Python-скрипт (`.py`) или ноутбук (`.ipynb`)** с вашим финальным кодом.
2.  **Финальный вывод агента** (скопированный структурированный текст или JSON).
3.  **Краткий анализ (1-2 абзаца) в файле `analysis.txt` или в Markdown-ячейке ноутбука**:
    *   Опишите, как вы изменили Pydantic-модель и промпт.
    *   Оцените, насколько хорошо агент справился с задачей. Нашел ли он все требования? Были ли ошибки или "галлюцинации"?
    *   Какие улучшения вы бы предложили для дальнейшей работы агента?